In [1]:
import cv2
import os
import glob
import numpy as np
import pandas as pd
import fnmatch
import time
import plotly.graph_objects as go
import plotly
import seaborn as sns
import matplotlib.pyplot as plt
import mahotas as mt
import scipy.stats as stats
import statsmodels.api as stm
import tensorflow as tf

In [2]:
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from scipy.stats import shapiro , normaltest , anderson , kstest, wilcoxon, kruskal
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
dry_201_path = 'D:/road/OPTICAL201_20210201230426314180_0.jpg'
dry_203_path = 'D:/road/OPTICAL203_20210201184719575368_.jpg'
wet_201_path = 'D:/road/OPTICAL201_20210201032754308899_1.jpg'
wet_203_path = 'D:/road/OPTICAL203_20210201060205825509_.jpg'

In [ ]:
path_list = [dry_201_path, dry_203_path, wet_201_path, wet_203_path]

In [ ]:
path_list

In [ ]:
roi = [(1, 14), (1, 15), (1, 16), (2, 14), (2, 15), (2, 16), (3, 14), (3, 15), (3, 16)]
height, width = 150, 150
eps = 1e-7

In [ ]:
haralick, lbp_list = [], []
for img_file in path_list:
    img = cv2.imread(img_file)
    for (i, j) in roi:
        x, y = i * width, j * height
        roi_img = img[y:y+height, x:x+width]
        gray_img = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)
        haralick_features = mt.features.haralick(gray_img).mean(axis=0)
        lbp = local_binary_pattern(gray_img, 8, 3, method='uniform')
        lbp = np.array(lbp)
        (hist, _) = np.histogram(lbp.ravel(), density=True, bins=int(lbp.max() + 1 ), range=(0, lbp.max() +1))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        
        haralick.append(haralick_features)
        lbp_list.append(hist)
        

In [ ]:
len(haralick)

In [ ]:
len(lbp_list)

## Haralick 테스트

In [84]:
source_path = 'D:/road'
categories = ['dry_201', 'dry_203', 'wet_201', 'wet_203']
width, height = 150, 150
eps = 1e-7
roi = [(1, 14), (1, 15), (1, 16), (2, 14), (2, 15), (2, 16), (3, 14), (3, 15), (3, 16)]
hal_col = ['hal1', 'hal2', 'hal3', 'hal4', 'hal5', 'hal6', 'hal7', 'hal8', 'hal9', 'hal10', 'hal11', 'hal12', 'hal13']
lbp_col = ['lbp1','lbp2','lbp3','lbp4','lbp5','lbp6','lbp7','lbp8','lbp9','lbp10']

In [4]:
for cat in categories:
    path = f'{source_path}/{cat}'
    if not os.path.isdir(path):
        print('Folder not Exist')
        exit()    
    haralick_list, lbp_list = [], []
    img_list = glob.glob(f'{path}/*jpg')
    
    for imgs in img_list:
        imgs = imgs.replace('\\', '/')
        img = cv2.imread(imgs)
        
        for (i, j) in roi:
            x, y = i * width, j * height
            roi_img = img[y:y+height, x:x+width]
            gray_img = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)
            
            haralick_feature = mt.features.haralick(gray_img).mean(axis=0)
            lbp_feature = local_binary_pattern(gray_img, 8, 3, method='uniform')
            lbp_feature = np.array(lbp_feature)
            (hist, _) = np.histogram(lbp_feature.ravel(), density=True, bins=int(lbp_feature.max() + 1), range=(0, int(lbp_feature.max() +1)))
            hist = hist.astype('float')
            hist /= (hist.sum() + eps)
            
            haralick_list.append(haralick_feature)
            lbp_list.append(hist)
            

    locals()[f'{cat}_haralick_pd'] = pd.DataFrame(np.array(haralick_list).reshape(-1, 13), columns=hal_col)
    locals()[f'{cat}_lbp_pd'] = pd.DataFrame(lbp_list, columns=lbp_col)
    

In [ ]:
mms = MinMaxScaler()
std = StandardScaler()

In [ ]:
dry_201_haralick_pd_std = mms.fit_transform(dry_201_haralick_pd)
wet_201_haralick_pd_std = mms.fit_transform(wet_201_haralick_pd)
dry_203_haralick_pd_std = mms.fit_transform(dry_203_haralick_pd)
wet_203_haralick_pd_std = mms.fit_transform(wet_203_haralick_pd)

In [ ]:
dry_201_haralick_pd_std = pd.DataFrame(dry_201_haralick_pd_std, columns=hal_col)
wet_201_haralick_pd_std = pd.DataFrame(wet_201_haralick_pd_std, columns=hal_col)
dry_203_haralick_pd_std = pd.DataFrame(dry_203_haralick_pd_std, columns=hal_col)
wet_203_haralick_pd_std = pd.DataFrame(wet_203_haralick_pd_std, columns=hal_col)

In [ ]:
def shapiro_test(df):
    for var in df.columns:
        stat, p = shapiro(df[var])
        hold = 0.05
        if p < hold:
            print(f'{var} : {p:.4f} 이므로 정규분포를 따르지 않는다')

In [ ]:
shapiro_test(wet_203_haralick_pd_std)

In [ ]:
shapiro(dry_203_haralick_pd)

In [ ]:
def cov_test(df1,df2):
    for x in df1.columns:
        cov = np.cov(df1[x], df2[x])
        cov = pd.DataFrame(cov)
        cov = cov.iloc[1:, :1]
        
        print(x, cov)

In [ ]:
def var_test(df1,df2):
    for x in df1.columns:
        p_value = stats.levene(df1[x], df2[x]).pvalue
        statistic = stats.levene(df1[x],df2[x]).statistic
        print(f'{x} = F_value : {statistic:.4f}, p_value: {p_value:.4f}')

In [ ]:
for col in dry_201_haralick_pd_std.columns:
    stat, p = kruskal(dry_201_haralick_pd_std[col], wet_201_haralick_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
for col in dry_203_haralick_pd_std.columns:
    stat, p = kruskal(dry_203_haralick_pd_std[col], wet_203_haralick_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
# dry_201_haralick_pd_std.drop(['hal2', 'hal4', 'hal5', 'hal6', 'hal7', 'hal9', 'hal10', 'hal11', 'hal12'], axis=1, inplace=True)
# wet_201_haralick_pd_std.drop(['hal2', 'hal4', 'hal5', 'hal6', 'hal7', 'hal9', 'hal10', 'hal11', 'hal12'], axis=1, inplace=True)

In [ ]:
dry_201_haralick_pd_std.drop(['hal2', 'hal12'], axis=1, inplace=True)
wet_201_haralick_pd_std.drop(['hal2', 'hal12'], axis=1, inplace=True)

In [ ]:
for col in dry_201_haralick_pd_std.columns:
    stat, p = wilcoxon(dry_201_haralick_pd_std[col], wet_201_haralick_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
# dry_203_haralick_pd_std.drop(['hal2', 'hal5', 'hal6', 'hal10', 'hal11', 'hal12'], axis=1, inplace=True)
# wet_203_haralick_pd_std.drop(['hal2', 'hal5', 'hal6', 'hal10', 'hal11', 'hal12'], axis=1, inplace=True)

In [ ]:
for col in dry_203_haralick_pd_std.columns:
    stat, p = wilcoxon(dry_203_haralick_pd_std[col], wet_203_haralick_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
# dry_203_haralick_pd.drop(['hal1'], axis=1, inplace=True)
# wet_203_haralick_pd.drop(['hal1'], axis=1, inplace=True)

In [6]:
dry_201_haralick_pd[['hal4', 'hal6', 'hal7']]

,hal4,hal6,hal7
0,71.916384,236.726489,264.212635
1,193.373913,259.095832,755.314511
2,89.373489,276.705816,343.831132
3,33.479860,291.813079,127.851841
4,13.231073,294.375930,47.135113
...,...,...,...
1795,17.187612,294.093925,64.766702
1796,45.762421,286.673409,177.538654
1797,86.146465,272.015117,334.078660
1798,33.748387,260.130243,123.481840


In [166]:
total_dry_haralick = pd.concat([dry_201_haralick_pd[['hal4', 'hal6', 'hal7']], dry_203_haralick_pd[['hal4', 'hal6', 'hal7']]], axis=1)
total_wet_haralick = pd.concat([wet_201_haralick_pd[['hal4', 'hal6', 'hal7']], wet_203_haralick_pd[['hal4', 'hal6', 'hal7']]], axis=1)

In [167]:
total_dry_haralick['result'] = 0
total_wet_haralick['result'] = 1

In [168]:
total_haralick = pd.concat([total_dry_haralick, total_wet_haralick], axis=0)

In [169]:
x_data, y_data = total_haralick.iloc[:, :-1], total_haralick.iloc[:, -1]

In [126]:
minmax = MinMaxScaler()
std=StandardScaler()

In [127]:
x_data = std.fit_transform(x_data)

In [170]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=2021, stratify=y_data)

In [171]:
svc = SVC()

In [172]:
params = {
    'C' : [x for x in range(0, 50)],
    'decision_function_shape' : ['ovo', 'ovr']
}

In [173]:
grid_svc = GridSearchCV(svc, params, cv=3, verbose=2)

In [174]:
grid_svc.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END ...................C=0, decision_function_shape=ovo; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovo; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovo; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovr; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovr; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovr; total time=   0.0s


C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py", line 226, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py", line 277, in _dense_fit
    self._probB, self.fit_status_ = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 192, in sklearn.svm._libsvm.fit
ValueError: C <= 0

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_valid

[CV] END ...................C=1, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovr; total time=   0.0s
[CV] END ...................

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.87746914 0.87746914 0.88611111 0.88611111
 0.88981481 0.88981481 0.89259259 0.89259259 0.89382716 0.89382716
 0.89537037 0.89537037 0.89753086 0.89753086 0.89876543 0.89876543
 0.90030864 0.90030864 0.90061728 0.90061728 0.90154321 0.90154321
 0.90216049 0.90216049 0.90277778 0.90277778 0.90401235 0.90401235
 0.90432099 0.90432099 0.90462963 0.90462963 0.90493827 0.90493827
 0.90617284 0.90617284 0.90679012 0.90679012 0.90740741 0.90740741
 0.90771605 0.90771605 0.90833333 0.90833333 0.9095679  0.9095679
 0.91018519 0.91018519 0.91111111 0.91111111 0.91141975 0.91141975
 0.9117284  0.9117284  0.91203704 0.91203704 0.91265432 0.91265432
 0.91388889 0.91388889 0.9132716  0.9132716  0.91419753 0.91419753
 0.91512346 0.91512346 0.9154321  0.9154321  0.91728395 0.91728395
 0.91759259 0.91759259 0.91790123 0.91790123 0

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                               15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                               27, 28, 29, ...],
                         'decision_function_shape': ['ovo', 'ovr']},
             verbose=2)

In [175]:
grid_svc.best_estimator_

SVC(C=48, decision_function_shape='ovo')

In [176]:
pred = grid_svc.best_estimator_.predict(x_test)

In [177]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95       180
           1       0.93      0.98      0.95       180

    accuracy                           0.95       360
   macro avg       0.95      0.95      0.95       360
weighted avg       0.95      0.95      0.95       360



In [178]:
test_path = r'D:/road/2102/2021-02-03'
# test_path = r'D:/road/test/'
# test_path = r'E:/MK-SD53R/2012/2020-12-27/202/fog_day'
# test_path = r'E:/new folder/test_fog/OPTICAL202_20201102081455231720_0.jpg'
# folder_list = os.listdir(paths)
count_0, count_1 = 0, 0
path_201 = f'{test_path}/201/wet_night'
path_203 = f'{test_path}/203/wet_night'
test_file_list_201 = glob.glob(f'{path_201}/*.jpg')
test_file_list_203 = glob.glob(f'{path_203}/*.jpg')

for img_201_path, img_203_path in zip(test_file_list_201, test_file_list_203):
    img_201 = cv2.imread(img_201_path)
    img_203 = cv2.imread(img_203_path)
    local_test = []
    for (i, j) in roi:
        x, y = i * width, j * height
        roi_img = img_201[y:y + height, x:x + width]
        roi_img_201 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_201 = mt.features.haralick(roi_img_201).mean(axis=0)
        lbp_201 = local_binary_pattern(roi_img_201, 8, 3, method='uniform')
        lbp_201 = np.array(lbp_201)
        (hist_201, _) = np.histogram(lbp_201.ravel(), density=True, bins=int(lbp_201.max() + 1),
                                 range=(0, int(lbp_201.max() + 1)))
        hist_201 = hist_201.astype("float")
        hist_201 /= (hist_201.sum() + eps)
        
        roi_img_203 = img_203[y:y + height, x:x + width]
        roi_img_203 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_203 = mt.features.haralick(roi_img_203).mean(axis=0)
        lbp_203 = local_binary_pattern(roi_img_203, 8, 3, method='uniform')
        lbp_203 = np.array(lbp_203)
        (hist_203, _) = np.histogram(lbp_203.ravel(), density=True, bins=int(lbp_203.max() + 1),
                                 range=(0, int(lbp_203.max() + 1)))
        hist_203 = hist_203.astype("float")
        hist_203 /= (hist_203.sum() + eps)
        
        concated = np.concatenate([np.delete(haralick_201,(0,1,2,4,7,8,9,10,11,12)), np.delete(haralick_203, (0,1,2,4,7,8,9,10,11,12))])
#         print(concated)
#         concated = minmax.fit_transform(concated)
#         concated = np.concatenate([haralick_201, haralick_203[:-1]])
        
        local_test.append(concated)
    
    local_test = np.array(local_test).reshape(-1, 6)
#     local_test = std.fit_transform(local_test)
    pred_values = grid_svc.best_estimator_.predict(local_test)
    preds = []
    for pred in pred_values:
        preds.append(pred)
    prediction_list = []
    for i in range(2):
        prediction_list.append(preds.count(i))
        
    if np.argmax(prediction_list) == 0:
        count_0 += 1
#         shutil.move(f'{file_path}', f'{test_path}/normal/{file_path.split("/")[-1]}')
        print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))
    if np.argmax(prediction_list) == 1:
        count_1 += 1
#         shutil.move(f'{file_path}', f'{test_path}/fog/{file_path.split("/")[-1]}')
        print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))    
        
print(f'건조 : {count_0/len(test_file_list_201)*100}%')
print(f'습윤 : {count_1/len(test_file_list_201)*100}%')

pred : [1, 0, 0, 0, 0, 0, 0, 0, 0] count : [8, 1] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 0, 0] count : [8, 1] class : 0
pred : [1, 0, 1, 0, 0, 0, 0, 0, 0] count : [7, 2] class : 0
pred : [1, 0, 1, 0, 0, 0, 0, 0, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 0, 0] count : [8, 1] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 0, 0] count : [8, 1] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 0, 0] count : [8, 1] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 0, 0, 0, 0, 0, 1, 0] count : [7, 2] class : 0
pred : [1, 0, 1, 0, 0, 0, 0, 0, 0] count

In [150]:
total_dry =  dry_203_haralick_pd - dry_201_haralick_pd
total_wet = wet_203_haralick_pd - wet_201_haralick_pd

In [151]:
total_dry['result'] = 0
total_wet['result'] = 1

In [152]:
total_haralick = pd.concat([total_dry, total_wet], axis=0)

In [153]:
x_data, y_data = total_haralick.iloc[:, :-1], total_haralick.iloc[:, -1]

In [154]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=2021, stratify=y_data)

In [155]:
svc = SVC()

In [156]:
params = {
    'C' : [x for x in range(0, 50)],
    'decision_function_shape' : ['ovo', 'ovr']
}

In [157]:
grid_svc = GridSearchCV(svc, params, cv=3, verbose=2)

In [158]:
grid_svc.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END ...................C=0, decision_function_shape=ovo; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovo; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovo; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovr; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovr; total time=   0.0s
[CV] END ...................C=0, decision_function_shape=ovr; total time=   0.0s
[CV] END ...................C=1, decision_function_shape=ovo; total time=   0.1s


C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py", line 226, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py", line 277, in _dense_fit
    self._probB, self.fit_status_ = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 192, in sklearn.svm._libsvm.fit
ValueError: C <= 0

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_valid

[CV] END ...................C=1, decision_function_shape=ovo; total time=   0.0s
[CV] END ...................C=1, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=1, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=2, decision_function_shape=ovr; total time=   0.1s
[CV] END ...................C=3, decision_function_shape=ovo; total time=   0.1s
[CV] END ...................

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.84814815 0.84814815 0.85709877 0.85709877
 0.86296296 0.86296296 0.86512346 0.86512346 0.86944444 0.86944444
 0.87191358 0.87191358 0.87345679 0.87345679 0.87592593 0.87592593
 0.87623457 0.87623457 0.87777778 0.87777778 0.87932099 0.87932099
 0.87993827 0.87993827 0.88055556 0.88055556 0.88024691 0.88024691
 0.8808642  0.8808642  0.88055556 0.88055556 0.8808642  0.8808642
 0.88148148 0.88148148 0.88148148 0.88148148 0.88148148 0.88148148
 0.88179012 0.88179012 0.88209877 0.88209877 0.88209877 0.88209877
 0.88179012 0.88179012 0.88148148 0.88148148 0.88148148 0.88148148
 0.88209877 0.88209877 0.88240741 0.88240741 0.88240741 0.88240741
 0.88271605 0.88271605 0.88395062 0.88395062 0.88364198 0.88364198
 0.88425926 0.88425926 0.88425926 0.88425926 0.8845679  0.8845679
 0.88487654 0.88487654 0.88487654 0.88487654 0.

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                               15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                               27, 28, 29, ...],
                         'decision_function_shape': ['ovo', 'ovr']},
             verbose=2)

In [159]:
grid_svc.best_estimator_

SVC(C=36, decision_function_shape='ovo')

In [160]:
pred = grid_svc.best_estimator_.predict(x_test)

In [161]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       180
           1       0.86      0.92      0.89       180

    accuracy                           0.89       360
   macro avg       0.89      0.89      0.89       360
weighted avg       0.89      0.89      0.89       360



In [162]:
x_train

,hal1,hal2,hal3,hal4,hal5,hal6,hal7,hal8,hal9,hal10,hal11,hal12,hal13
1124,0.004595,-1.347866,0.017777,-2.083860,0.117367,-10.696204,-6.987572,-0.063332,-0.517333,0.000454,-0.302163,-0.091778,0.010783
1214,-0.001535,0.308039,0.104695,7.378149,0.016411,-1.283071,29.204558,0.295211,0.254501,0.000010,-0.023331,-0.053035,0.079558
289,0.001633,-5.340583,0.103986,-0.963416,0.069145,-1.948508,1.486918,0.006239,-0.363813,0.000287,-0.335811,-0.072922,0.070356
619,0.004852,-28.055021,-0.052361,-55.450148,0.135365,-6.484160,-193.745570,-0.863199,-1.615685,0.000470,-0.731775,-0.006990,-0.023972
360,0.003621,-12.874433,0.265676,0.824624,0.172040,0.464155,16.172929,0.129825,-0.689468,0.000694,-0.724212,-0.178554,0.164411
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.016440,-8.657063,0.005870,-27.934922,0.197841,11.537297,-103.082626,-0.904728,-1.888879,0.000698,-0.825462,-0.076030,0.007738
84,0.004500,-3.634702,-0.062916,-14.744577,0.082207,-5.948115,-55.343606,-0.410115,-0.781351,0.000316,-0.299220,-0.010915,-0.007955
1433,0.008632,-6.496397,-0.013754,-54.881570,0.104697,13.373570,-213.029882,-0.775379,-1.362008,0.000563,-0.557751,-0.018438,-0.007712
78,0.005445,-10.849467,-0.055033,-157.914941,0.137861,0.483582,-620.810299,-0.979943,-1.672065,0.000723,-0.744175,-0.010699,-0.010671


In [164]:
test_path = r'D:/road/2102/2021-02-03'
# test_path = r'D:/road/test/'
# test_path = r'E:/MK-SD53R/2012/2020-12-27/202/fog_day'
# test_path = r'E:/new folder/test_fog/OPTICAL202_20201102081455231720_0.jpg'
# folder_list = os.listdir(paths)
count_0, count_1 = 0, 0
# path_201 = f'{test_path}/201'
# path_203 = f'{test_path}/203'
path_201 = f'{test_path}/201/wet_night'
path_203 = f'{test_path}/203/wet_night'
test_file_list_201 = glob.glob(f'{path_201}/*.jpg')
test_file_list_203 = glob.glob(f'{path_203}/*.jpg')

for img_201_path, img_203_path in zip(test_file_list_201, test_file_list_203):
    img_201 = cv2.imread(img_201_path)
    img_203 = cv2.imread(img_203_path)
    local_test = []
    for (i, j) in roi:
        x, y = i * width, j * height
        
        roi_img= img_201[y:y + height, x:x + width]
        roi_img_201 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_201 = mt.features.haralick(roi_img_201).mean(axis=0)
        lbp_201 = local_binary_pattern(roi_img_201, 8, 3, method='uniform')
        lbp_201 = np.array(lbp_201)
        (hist_201, _) = np.histogram(lbp_201.ravel(), density=True, bins=int(lbp_201.max() + 1),
                                 range=(0, int(lbp_201.max() + 1)))
        hist_201 = hist_201.astype("float")
        hist_201 /= (hist_201.sum() + eps)
        
        roi_img_203 = img_203[y:y + height, x:x + width]
        roi_img_203 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_203 = mt.features.haralick(roi_img_203).mean(axis=0)
        lbp_203 = local_binary_pattern(roi_img_203, 8, 3, method='uniform')
        lbp_203 = np.array(lbp_203)
        (hist_203, _) = np.histogram(lbp_203.ravel(), density=True, bins=int(lbp_203.max() + 1),
                                 range=(0, int(lbp_203.max() + 1)))
        hist_203 = hist_203.astype("float")
        hist_203 /= (hist_203.sum() + eps)
        
        concated = haralick_203 - haralick_201
#         concated = np.concatenate([np.delete(haralick_201,(0,1,2,4,7,8,9,10,11,12)), np.delete(haralick_203, (0,1,2,4,7,8,9,10,11,12))])
#         concated = mms.fit_transform(concated)
#         concated = np.concatenate([haralick_201, haralick_203[:-1]])
        
        local_test.append(concated)
    
    local_test = np.array(local_test).reshape(-1, 13)
#     local_test = std.fit_transform(local_test)
    pred_values = grid_svc.best_estimator_.predict(local_test)
    preds = []
    for pred in pred_values:
        preds.append(pred)
    prediction_list = []
    for i in range(2):
        prediction_list.append(preds.count(i))
        
    if np.argmax(prediction_list) == 0:
        count_0 += 1
#         shutil.move(f'{file_path}', f'{test_path}/normal/{file_path.split("/")[-1]}')
        print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))
    if np.argmax(prediction_list) == 1:
        count_1 += 1
#         shutil.move(f'{file_path}', f'{test_path}/fog/{file_path.split("/")[-1]}')
        print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))    
        
print(f'건조 : {count_0/len(test_file_list_201)*100}%')
print(f'습윤 : {count_1/len(test_file_list_201)*100}%')

pred : [0, 1, 0, 0, 0, 0, 0, 0, 1] count : [7, 2] class : 0
pred : [0, 0, 0, 0, 0, 1, 0, 0, 0] count : [8, 1] class : 0
pred : [0, 0, 0, 0, 0, 1, 0, 0, 1] count : [7, 2] class : 0
pred : [0, 1, 0, 1, 0, 1, 1, 1, 1] count : [3, 6] class : 1
pred : [0, 1, 0, 0, 1, 1, 0, 1, 1] count : [4, 5] class : 1
pred : [0, 1, 0, 0, 1, 1, 0, 1, 1] count : [4, 5] class : 1
pred : [0, 0, 1, 0, 1, 1, 0, 1, 1] count : [4, 5] class : 1
pred : [0, 0, 1, 0, 1, 1, 0, 1, 1] count : [4, 5] class : 1
pred : [0, 1, 0, 1, 1, 1, 0, 0, 1] count : [4, 5] class : 1
pred : [1, 0, 0, 0, 1, 1, 1, 1, 1] count : [3, 6] class : 1
pred : [0, 1, 0, 1, 1, 1, 0, 1, 1] count : [3, 6] class : 1
pred : [0, 0, 0, 0, 1, 1, 0, 1, 0] count : [6, 3] class : 0
pred : [1, 1, 0, 1, 1, 1, 0, 1, 1] count : [2, 7] class : 1
pred : [0, 1, 0, 1, 1, 1, 0, 1, 1] count : [3, 6] class : 1
pred : [1, 1, 0, 1, 1, 1, 0, 1, 1] count : [2, 7] class : 1
pred : [0, 1, 0, 0, 1, 1, 0, 0, 0] count : [6, 3] class : 0
pred : [1, 0, 0, 0, 1, 1, 0, 1, 1] count

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [ ]:
model = Sequential([
    Dense(512, activation='relu', input_shape=x_train[0].shape),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dropout(0.1),
    Dense(8, activation='relu'),
    Dense(2, activation='softmax')
    
])

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
ct = tf.keras.callbacks.ModelCheckpoint('201_203_h_200.tf', save_best_only=True)

In [ ]:
history = model.fit(x_train, y_train, epochs=1000, batch_size=128, validation_data=(x_test, y_test), callbacks=[ct])

In [ ]:
load_model = tf.keras.models.load_model('201_203_h_200.tf')

In [ ]:
# test_path = r'D:/road/2102/2021-02-03'
test_path = r'D:/road/test/'
# test_path = r'E:/MK-SD53R/2012/2020-12-27/202/fog_day'
# test_path = r'E:/new folder/test_fog/OPTICAL202_20201102081455231720_0.jpg'
# folder_list = os.listdir(paths)
count_0, count_1 = 0, 0
path_201 = f'{test_path}/201/'
path_203 = f'{test_path}/203/'
test_file_list_201 = glob.glob(f'{path_201}/*.jpg')
test_file_list_203 = glob.glob(f'{path_203}/*.jpg')

for img_201_path, img_203_path in zip(test_file_list_201, test_file_list_203):
    img_201 = cv2.imread(img_201_path)
    img_203 = cv2.imread(img_203_path)
    local_test = []
    for (i, j) in roi:
        x, y = x * width, y * height
        
        roi_img_201 = img_201[y:y + height, x:x + width]
        roi_img_201 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_201 = mt.features.haralick(roi_img_201).mean(axis=0)
        lbp_201 = local_binary_pattern(roi_img_201, 8, 3, method='uniform')
        lbp_201 = np.array(lbp_201)
        (hist_201, _) = np.histogram(lbp_201.ravel(), density=True, bins=int(lbp_201.max() + 1),
                                 range=(0, int(lbp_201.max() + 1)))
        hist_201 = hist_201.astype("float")
        hist_201 /= (hist_201.sum() + eps)
        
        roi_img_203 = img_203[y:y + height, x:x + width]
        roi_img_203 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_203 = mt.features.haralick(roi_img_203).mean(axis=0)
        lbp_203 = local_binary_pattern(roi_img_203, 8, 3, method='uniform')
        lbp_203 = np.array(lbp_203)
        (hist_203, _) = np.histogram(lbp_203.ravel(), density=True, bins=int(lbp_203.max() + 1),
                                 range=(0, int(lbp_203.max() + 1)))
        hist_203 = hist_203.astype("float")
        hist_203 /= (hist_203.sum() + eps)
        
        concated = np.concatenate([np.delete(haralick_201,(0, 1)), np.delete(haralick_201,(0, 1, 10))])
#         concated = np.concatenate([haralick_201, haralick_203[:-1]])
        
        local_test.append(concated)
        
    local_test = np.array(local_test).reshape(-1, 21)
    pred_values = load_model.predict(local_test)
    
    preds = []
    for pred in pred_values:
        preds.append(np.argmax(pred))
    prediction_list = []
    for i in range(2):
        prediction_list.append(preds.count(i))
        
#     print()
    if np.argmax(prediction_list) == 0:
        count_0 += 1
#         shutil.move(f'{file_path}', f'{test_path}/normal/{file_path.split("/")[-1]}')
#         print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))
    if np.argmax(prediction_list) == 1:
        count_1 += 1
#         shutil.move(f'{file_path}', f'{test_path}/fog/{file_path.split("/")[-1]}')
#         print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))    
        
print(f'건조 : {count_0/len(test_file_list_201)*100}%')
print(f'습윤 : {count_1/len(test_file_list_201)*100}%')


#     preds = []
#     for pred in pred_values:
#         preds.append(pred)
#     prediction_list = []
#     for i in range(2):
#         prediction_list.append(preds.count(i))
        
#     if np.argmax(prediction_list) == 0:
#         count_0 += 1
# #         shutil.move(f'{file_path}', f'{test_path}/normal/{file_path.split("/")[-1]}')
#         print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))
#     if np.argmax(prediction_list) == 1:
#         count_1 += 1
# #         shutil.move(f'{file_path}', f'{test_path}/fog/{file_path.split("/")[-1]}')
#         print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))    
        
# print(f'건조 : {count_0/len(test_file_list_201)*100}%')
# print(f'습윤 : {count_1/len(test_file_list_201)*100}%')

In [ ]:
y_train

## LBP 테스트

In [ ]:
dry_201_lbp_pd_std = mms.fit_transform(dry_201_lbp_pd)
wet_201_lbp_pd_std = mms.fit_transform(wet_201_lbp_pd)
dry_203_lbp_pd_std = mms.fit_transform(dry_203_lbp_pd)
wet_203_lbp_pd_std = mms.fit_transform(wet_203_lbp_pd)

In [ ]:
dry_201_lbp_pd_std = pd.DataFrame(dry_201_lbp_pd_std, columns=lbp_col)
wet_201_lbp_pd_std = pd.DataFrame(wet_201_lbp_pd_std, columns=lbp_col)
dry_203_lbp_pd_std = pd.DataFrame(dry_203_lbp_pd_std, columns=lbp_col)
wet_203_lbp_pd_std = pd.DataFrame(wet_203_lbp_pd_std, columns=lbp_col)

In [ ]:
for col in dry_201_lbp_pd_std.columns:
    stat, p = kruskal(dry_201_lbp_pd_std[col], wet_201_lbp_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
for col in dry_201_lbp_pd_std.columns:
    stat, p = wilcoxon(dry_201_lbp_pd_std[col], wet_201_lbp_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
for col in dry_203_lbp_pd_std.columns:
    stat, p = kruskal(dry_203_lbp_pd_std[col], wet_203_lbp_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
for col in dry_203_lbp_pd_std.columns:
    stat, p = wilcoxon(dry_203_lbp_pd_std[col], wet_203_lbp_pd_std[col])
    print(f'{col} : {p:.4f}')

In [ ]:
# dry_201_lbp_pd.drop(['lbp1', 'lbp6', 'lbp7'], axis=1, inplace=True)
# wet_201_lbp_pd.drop(['lbp1', 'lbp6', 'lbp7'], axis=1, inplace=True)

In [ ]:
dry_203_lbp_pd_std.drop(['lbp5', 'lbp7'], axis=1, inplace=True)
wet_203_lbp_pd_std.drop(['lbp5', 'lbp7'], axis=1, inplace=True)

In [ ]:
total_dry_lbp = pd.concat([dry_201_lbp_pd_std, dry_203_lbp_pd_std], axis=1)
total_wet_lbp = pd.concat([wet_201_lbp_pd_std, wet_203_lbp_pd_std], axis=1)

In [ ]:
total_dry_lbp['result'] = 0
total_wet_lbp['result'] = 1

In [ ]:
total_lbp = pd.concat([total_dry_lbp, total_wet_lbp], axis=0)

In [ ]:
x_data, y_data = total_lbp.iloc[:, :-1], total_lbp.iloc[:, -1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=2021, stratify=y_data)

In [ ]:
svc = SVC()

In [ ]:
params = {
    'C' : [x for x in range(0, 50)],
    'decision_function_shape' : ['ovo', 'ovr']
}

In [ ]:
grid_svc = GridSearchCV(svc, params, cv=3, verbose=2)

In [ ]:
grid_svc.fit(x_train, y_train)

In [ ]:
grid_svc.best_estimator_

In [ ]:
pred = grid_svc.best_estimator_.predict(x_test)

In [ ]:
len(x_train.columns)

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
test_path = r'D:/road/2102/2021-02-03'
# test_path = r'D:/road/test/'
# test_path = r'E:/MK-SD53R/2012/2020-12-27/202/fog_day'
# test_path = r'E:/new folder/test_fog/OPTICAL202_20201102081455231720_0.jpg'
# folder_list = os.listdir(paths)
count_0, count_1 = 0, 0
path_201 = f'{test_path}/201/wet_night'
path_203 = f'{test_path}/203/wet_night'
test_file_list_201 = glob.glob(f'{path_201}/*.jpg')
test_file_list_203 = glob.glob(f'{path_203}/*.jpg')

for img_201_path, img_203_path in zip(test_file_list_201, test_file_list_203):
    img_201 = cv2.imread(img_201_path)
    img_203 = cv2.imread(img_203_path)
    local_test = []
    for (i, j) in roi:
        x, y = x * width, y * height
        
        roi_img_201 = img_201[y:y + height, x:x + width]
        roi_img_201 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_201 = mt.features.haralick(roi_img_201).mean(axis=0)
        lbp_201 = local_binary_pattern(roi_img_201, 8, 3, method='uniform')
        lbp_201 = np.array(lbp_201)
        (hist_201, _) = np.histogram(lbp_201.ravel(), density=True, bins=int(lbp_201.max() + 1),
                                 range=(0, int(lbp_201.max() + 1)))
        hist_201 = hist_201.astype("float")
        hist_201 /= (hist_201.sum() + eps)
        
        roi_img_203 = img_203[y:y + height, x:x + width]
        roi_img_203 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_203 = mt.features.haralick(roi_img_203).mean(axis=0)
        lbp_203 = local_binary_pattern(roi_img_203, 8, 3, method='uniform')
        lbp_203 = np.array(lbp_203)
        (hist_203, _) = np.histogram(lbp_203.ravel(), density=True, bins=int(lbp_203.max() + 1),
                                 range=(0, int(lbp_203.max() + 1)))
        hist_203 = hist_203.astype("float")
        hist_203 /= (hist_203.sum() + eps)
        
        concated = np.concatenate([hist_201, np.delete(hist_203, (4, 6))])
#         concated = np.concatenate([haralick_201, haralick_203[:-1]])
        
        local_test.append(concated)
        
    local_test = np.array(local_test).reshape(-1, 18)
    local_test = mms.fit_transform(local_test)
    pred_values = grid_svc.best_estimator_.predict(local_test)
    preds = []
    for pred in pred_values:
        preds.append(pred)
    prediction_list = []
    for i in range(5):
        prediction_list.append(preds.count(i))
        
    if np.argmax(prediction_list) == 0:
        count_0 += 1
#         shutil.move(f'{file_path}', f'{test_path}/normal/{file_path.split("/")[-1]}')
        print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))
    if np.argmax(prediction_list) == 1:
        count_1 += 1
#         shutil.move(f'{file_path}', f'{test_path}/fog/{file_path.split("/")[-1]}')
        print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))    
        
print(f'건조 : {count_0/len(test_file_list_201)*100}%')
print(f'습윤 : {count_1/len(test_file_list_201)*100}%')

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [ ]:
try:
    del model
except:
    pass

In [ ]:
model = Sequential([
    Dense(512, activation='relu', input_shape=x_train[0].shape),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dropout(0.1),
    Dense(8, activation='relu'),
    Dense(2, activation='softmax')
    
])

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
ct = tf.keras.callbacks.ModelCheckpoint('201_203_l.tf', save_best_only=True)

In [ ]:
history = model.fit(x_train, y_train, epochs=1000, batch_size=128, validation_data=(x_test, y_test), callbacks=[ct])

In [ ]:
load_model = tf.keras.models.load_model('201_203_l.tf/')

In [ ]:
test_path = r'D:/road/2102/2021-02-03'
# test_path = r'D:/road/test/'
# test_path = r'E:/MK-SD53R/2012/2020-12-27/202/fog_day'
# test_path = r'E:/new folder/test_fog/OPTICAL202_20201102081455231720_0.jpg'
# folder_list = os.listdir(paths)
count_0, count_1 = 0, 0
path_201 = f'{test_path}/201/wet_night'
path_203 = f'{test_path}/203/wet_night'
test_file_list_201 = glob.glob(f'{path_201}/*.jpg')
test_file_list_203 = glob.glob(f'{path_203}/*.jpg')

for img_201_path, img_203_path in zip(test_file_list_201, test_file_list_203):
    img_201 = cv2.imread(img_201_path)
    img_203 = cv2.imread(img_203_path)
    local_test = []
    for (i, j) in roi:
        x, y = x * width, y * height
        
        roi_img_201 = img_201[y:y + height, x:x + width]
        roi_img_201 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_201 = mt.features.haralick(roi_img_201).mean(axis=0)
        lbp_201 = local_binary_pattern(roi_img_201, 8, 3, method='uniform')
        lbp_201 = np.array(lbp_201)
        (hist_201, _) = np.histogram(lbp_201.ravel(), density=True, bins=int(lbp_201.max() + 1),
                                 range=(0, int(lbp_201.max() + 1)))
        hist_201 = hist_201.astype("float")
        hist_201 /= (hist_201.sum() + eps)
        
        roi_img_203 = img_203[y:y + height, x:x + width]
        roi_img_203 = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)  # 그레이 스케일
        haralick_203 = mt.features.haralick(roi_img_203).mean(axis=0)
        lbp_203 = local_binary_pattern(roi_img_203, 8, 3, method='uniform')
        lbp_203 = np.array(lbp_203)
        (hist_203, _) = np.histogram(lbp_203.ravel(), density=True, bins=int(lbp_203.max() + 1),
                                 range=(0, int(lbp_203.max() + 1)))
        hist_203 = hist_203.astype("float")
        hist_203 /= (hist_203.sum() + eps)
        
        concated = np.concatenate([hist_201, np.delete(hist_203, (4, 6))])
#         concated = np.concatenate([haralick_201, haralick_203[:-1]])
        
        local_test.append(concated)
        
    local_test = np.array(local_test).reshape(-1, 18)
    local_test = mms.fit_transform(local_test)
    pred_values = load_model.predict(local_test)
    
    preds = []
    for pred in pred_values:
        preds.append(np.argmax(pred))
    prediction_list = []
    for i in range(2):
        prediction_list.append(preds.count(i))
        
#     print()
    if np.argmax(prediction_list) == 0:
        count_0 += 1
#         shutil.move(f'{file_path}', f'{test_path}/normal/{file_path.split("/")[-1]}')
#         print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))
    if np.argmax(prediction_list) == 1:
        count_1 += 1
#         shutil.move(f'{file_path}', f'{test_path}/fog/{file_path.split("/")[-1]}')
#         print('pred :', preds, 'count :', prediction_list, 'class :', np.argmax(prediction_list))    
        
print(f'건조 : {count_0/len(test_file_list_201)*100}%')
print(f'습윤 : {count_1/len(test_file_list_201)*100}%')
